##  Calculate Yearly DPLE variables

In [1]:
import xarray as xr
import numpy as np
import metabolic as mi
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')

In [2]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """

    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)

In [3]:
USER = os.environ['USER']

In [69]:
#varr = 'O2'
#varr = 'AOU'
#varr = 'TEMP'
varr = 'SALT'

In [70]:
path = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{path}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly.nc'
filename = f'{path}/DPLE_driftcorrected_Phi_components_{varr}_vertical_mean_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)

In [71]:
dsf = xr.Dataset()
for cc in range(3):
    print(cc)
    for layer in ['0-200m', '200-600m']:
        dsf[f'Phi_{varr}_{cc}_{layer}'] = xr_reshape(ds[f'Phi_{varr}_{cc}_{layer}'][2:,:], 'L', ['year', 'mon'], [np.arange(10), np.arange(12)]).mean(dim='mon')

0
1
2


In [72]:
for cc in range(3):
    for layer in ['0-200m', '200-600m']:
        dsf[f'Phi_{varr}_{cc}_{layer}'] = dsf[f'Phi_{varr}_{cc}_{layer}'].transpose("Y", "year", "nlat", "nlon")

In [73]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:              (nlat: 140, nlon: 195, Y: 64, year: 10)
Coordinates:
    ULONG                (nlat, nlon) float64 101.7 102.9 104.0 ... 319.5 320.0
    ULAT                 (nlat, nlon) float64 15.13 15.13 15.14 ... 70.63 70.63
  * Y                    (Y) int64 1954 1955 1956 1957 ... 2014 2015 2016 2017
    TLAT                 (nlat, nlon) float64 14.96 14.96 14.97 ... 70.42 70.42
    TLONG                (nlat, nlon) float64 101.2 102.3 103.4 ... 319.2 319.7
  * year                 (year) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_SALT_0_0-200m    (Y, year, nlat, nlon) float64 nan nan nan ... nan nan
    Phi_SALT_0_200-600m  (Y, year, nlat, nlon) float64 nan nan nan ... nan nan
    Phi_SALT_1_0-200m    (Y, year, nlat, nlon) float64 nan nan nan ... nan nan
    Phi_SALT_1_200-600m  (Y, year, nlat, nlon) float64 nan nan nan ... nan nan
    Phi_SALT_2_0-200m    (Y, year, nlat, nlon) float64 nan nan nan ... nan nan
    Phi_SALT_2_200-600m  (Y, year, nlat, nlon) float64 nan nan nan ... nan nan

In [74]:
#dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly_to_yearly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_components_{varr}_vertical_mean_ens_mean_monthly_to_yearly.nc', mode='w')

## LME monthly to yearly

In [45]:
path = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{path}/DPLE_driftcorrected_Phi_LME_ens_mean_monthly.nc'
filename = f'{path}/DPLE_driftcorrected_Phi_components_{varr}_LME_ens_mean_monthly.nc'
ds = xr.open_dataset(filename)
ds

<xarray.Dataset>
Dimensions:              (lme: 11, L: 122, Y: 64)
Coordinates:
  * lme                  (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * L                    (L) int64 1 2 3 4 5 6 7 ... 116 117 118 119 120 121 122
  * Y                    (Y) int64 1954 1955 1956 1957 ... 2014 2015 2016 2017
Data variables:
    Phi_SALT_0_0-200m    (lme, L, Y) float64 ...
    Phi_SALT_0_200-600m  (lme, L, Y) float64 ...
    Phi_SALT_1_0-200m    (lme, L, Y) float64 ...
    Phi_SALT_1_200-600m  (lme, L, Y) float64 ...
    Phi_SALT_2_0-200m    (lme, L, Y) float64 ...
    Phi_SALT_2_200-600m  (lme, L, Y) float64 ...

In [46]:
dsf = xr.Dataset()
for cc in range(3):
    print(cc)
    for layer in ['0-200m', '200-600m']:
        dsf[f'Phi_{varr}_{cc}_{layer}'] = xr_reshape(ds[f'Phi_{varr}_{cc}_{layer}'][:, 2:, :], 'L', ['year', 'mon'], [np.arange(10), np.arange(12)]).mean(dim='mon')

0
1
2


In [47]:
for cc in range(3):
    for layer in ['0-200m', '200-600m']:
        dsf[f'Phi_{varr}_{cc}_{layer}'] = dsf[f'Phi_{varr}_{cc}_{layer}'].transpose("lme","Y", "year")

In [48]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:              (lme: 11, Y: 64, year: 10)
Coordinates:
  * lme                  (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * Y                    (Y) int64 1954 1955 1956 1957 ... 2014 2015 2016 2017
  * year                 (year) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    Phi_SALT_0_0-200m    (lme, Y, year) float64 3.773e-05 ... -0.0003346
    Phi_SALT_0_200-600m  (lme, Y, year) float64 -2.753e-05 ... -3.449e-06
    Phi_SALT_1_0-200m    (lme, Y, year) float64 4.86e-05 ... -0.0004025
    Phi_SALT_1_200-600m  (lme, Y, year) float64 -3.405e-05 ... -4.253e-06
    Phi_SALT_2_0-200m    (lme, Y, year) float64 7.272e-05 ... -0.0005403
    Phi_SALT_2_200-600m  (lme, Y, year) float64 -4.781e-05 ... -5.941e-06

In [49]:
#dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_LME_ens_mean_monthly_to_yearly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_components_{varr}_LME_ens_mean_monthly_to_yearly.nc', mode='w')